In [1]:
import gmaps
from API_data import gmap_key
import pandas as pd
import requests
import numpy as np
gmaps.configure(api_key=gmap_key)
city_data_df = pd.read_csv("./city_output_data/city_data.csv")


In [3]:
# humidity heat map
locations = city_data_df[["Lat","Lon"]]
humidity = city_data_df["Humidity"]

fig = gmaps.figure(zoom_level=1,center=(0,0))
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# filter data for ideal weather conditions
filtered_data = city_data_df.loc[city_data_df["Temp"] > 67]
filtered_data = filtered_data.loc[filtered_data["Temp"] < 75]
filtered_data = filtered_data.loc[filtered_data["Wind Speed"] < 5]
filtered_data = filtered_data.loc[filtered_data["Humidity"] < 60]
print(filtered_data["City"].count())
filtered_data

7


,City Number,City,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed,Country
33,33,alta floresta,-9.8756,-56.0861,74.55,53,0,3.27,BR
39,39,sabathu,30.9719,76.9933,72.01,52,12,2.46,IN
111,111,taksimo,56.3444,114.8794,68.86,56,90,3.76,RU
167,167,faya,18.3851,42.4509,69.80,52,100,4.47,SA
330,330,bageshwar,29.8500,79.7667,73.45,53,4,1.28,IN
353,353,vilhena,-12.7406,-60.1458,74.66,49,73,3.44,BR
508,508,khandyga,62.6667,135.6000,72.43,44,0,3.69,RU


In [5]:
hotel_df = filtered_data[["City","Lat","Lon","Country"]]
hotel_df["Hotel"]= ""
hotel_df.index = np.arange(1, len(hotel_df) + 1)
hotel_df

D:\Users\9csic\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lon,Country,Hotel
1,alta floresta,-9.8756,-56.0861,BR,
2,sabathu,30.9719,76.9933,IN,
3,taksimo,56.3444,114.8794,RU,
4,faya,18.3851,42.4509,SA,
5,bageshwar,29.8500,79.7667,IN,
6,vilhena,-12.7406,-60.1458,BR,
7,khandyga,62.6667,135.6000,RU,


In [6]:
radius = 5000
search_type = "hotel"
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input="+ search_type + "&inputtype=textquery&fields=photos,formatted_address,name,opening_hours,rating&locationbias=circle:5000@"

for index, row in hotel_df.iterrows():
    url = f'{base_url}{row["Lat"]},{row["Lon"]}&key={gmap_key}'
    response = requests.get(url).json()
    hotel_df.loc[index, "Hotel"] = (response["candidates"][0]["name"])
    
hotel_df

D:\Users\9csic\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Lat,Lon,Country,Hotel
1,alta floresta,-9.8756,-56.0861,BR,Hotel Mandino
2,sabathu,30.9719,76.9933,IN,Baba Resorts
3,taksimo,56.3444,114.8794,RU,"Gostinitsa ""Viktoriya"""
4,faya,18.3851,42.4509,SA,Abha Palace Hotel
5,bageshwar,29.8500,79.7667,IN,Mystic Mountain Hotel
6,vilhena,-12.7406,-60.1458,BR,Hotel Mirage
7,khandyga,62.6667,135.6000,RU,Prezidentskaya Gostinitsa


In [7]:
hotel_coordinates = hotel_df[["Lat", "Lon"]]
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
label_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


hotel_layer = gmaps.marker_layer(hotel_coordinates,info_box_content=label_info)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))